In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import sys

# Oregon 2020

## VEST Documentation

Election results from the Oregon Secretary of State via OpenElections (https://github.com/openelections/openelections-data-or/). Results from Polk, Tillamook, Wallowa, and Wasco counties were corrected with the respective county canvass reports.

Precinct shapefiles for the following counties were provided by the respective county governments: Benton, Clatsop, Columbia, Coos, Crook, Curry, Deschutes, Douglas, Harney, Hood River, Jackson, Josephine, Klamath, Lane, Linn, Malheur, Morrow, Multnomah, Polk, Sherman, Tillamook, Umatilla, Union, Wallowa, Wasco, Yamhill.

Precinct shapefiles for the following counties were provided by the Oregon Secretary of State: Jefferson, Lane, Marion. Several gaps in the Marion County shapefile were assigned based on the Precinct Split Summary Report.

Precinct shapefiles for Clackamas and Washington were provided by the Metro Regional Government Data Resource Center.

Precinct boundaries for Grant County and Wheeler County are defined by school district boundaries. Precinct shapefiles were produced to match PDF maps provided by the respective counties using the 2016 Oregon Education Boundaries shapefile obtained from the Oregon Spatial Data Library.

Precinct shapefiles for the following counties were produced based on PDF maps provided by the respective counties: Baker, Gilliam, Lake, Lincoln. Boundaries defined by the PLSS grid were produced with the Oregon Public Land Survey Quarter-Quarter Reference Grid shapefile obtained from the Oregon Spatial Data Library. Municipal boundaries were produced from the Oregon Department of Transportion 2016 City Limits shapefile obtained from the Oregon Spatial Data Library. Street boundaries were produced from the U.S. Census Bureau census block shapefiles.

PDF maps obtained from Lake County are approximate precinct boundaries drawn on the PLSS grid for county precincts and on the street map for Lakeview city precincts. These boundaries were adjusted to match address range assignments in the Lake County Address Library Report which serves as the legal definition of the precincts according to the Lake County Clerk. Precinct divisions across roads that span multiple rural precincts are generally defined by zip codes. Address ranges were further identified based on the Lake County tax lot parcel viewer.

Rural precinct boundaries in the shapefiles provided by Coos County and Union County were revised to match the updated PLSS cadastral grid in the Oregon GIS Framework obtained from the Oregon Spatial Data Library. 

Municipal precinct boundaries in the following counties were edited to match city limits in effect for the November 2020 general election: Coos, Deschutes, Douglas, Jefferson, Malheur, Marion, Tillamook, Umatilla, Washington, Yamhill.

Precinct numbers in the Columbia County, Josephine County, and Wallowa County shapefiles were edited to match the Oregon Secretary of State voter file.

G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRERTRU - Donald J. Trump (Republican Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREGHAW - Howie Hawkins (Pacific Green Party)  
G20PREPHUN - Dario Hunter (Progressive Party)  
G20PREOWRI - Write-in Votes  

G20USSDMER - Jeff Merkley (Democratic Party, Independent Party and Working Family Party (fusion candidate))  
G20USSRPER - Jo Rae Perkins (Republican Party)  
G20USSLDYE - Gary Dye (Libertarian Party)  
G20USSGTAH - Ibrahim A. Taher (Pacific Green Party and Progressive Party (fusion candidate))  
G20USSOWRI - Write-in Votes  

G20ATGDROS - Ellen Rosenblum (Democratic Party, Independent Party and Working Family Party (fusion candidate))  
G20ATGRCRO - Michael Cross (Republican Party)  
G20ATGLHED - Lars D. H. Hedbor (Libertarian Party)  
G20ATGOWRI - Write-in Votes  

G20SOSDFAG - Shemia Fagan (Democratic Party and Working Family Party (fusion candidate))  
G20SOSRTHA - Kim Thatcher (Republican Party and Independent Party (fusion candidate))  
G20SOSLMAR - Kyle Markley (Libertarian Party)  
G20SOSGPAR - Nathalie Paravicini (Pacific Green Party and Progressive Party (fusion candidate))  
G20SOSOWRI - Write-in Votes  

G20TREDREA - Tobias Read (Democratic Party and Working Family Party (fusion candidate))  
G20TRERGUD - Jeff Gudman (Republican Party)  
G20TREIHEN - Chris Henry (Independent Party, Pacific Green Party and Progressive Party (fusion candidate))  
G20TRECMAR - Michael P. Marsh (Constitution Party)  
G20TREOWRI - Write-in Votes  

## Load VEST File

In [2]:
vest_or_20 = gp.read_file("./raw-from-source/VEST/or_2020/or_2020.shp")

In [3]:
data_columns = [x for x in vest_or_20.columns if "G20" in x]

## Load OR Votes

In [4]:
file_list = os.listdir("./raw-from-source/Election_Results/2020/counties")
li = []
for file in file_list:
    #print(file)
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/Election_Results/2020/counties/"+file
        file_prev = pd.read_csv(file_ref)
        if 'precincts' in file_prev.columns:
            file_prev['precinct'] = file_prev['precincts']
     
        file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
        li.append(file_prev)
election_results = pd.concat(li, axis=0, ignore_index=True)
display(election_results)

,county,precinct,office,district,party,candidate,votes
0,Linn,Precinct 001,President,NaN,REP,Donald J. Trump,208.0
1,Linn,Precinct 001,President,NaN,DEM,Joseph R. Biden,428.0
2,Linn,Precinct 001,President,NaN,LBT,Jo Jorgensen,11.0
3,Linn,Precinct 001,President,NaN,PGP,Howie Hawkins,8.0
4,Linn,Precinct 001,President,NaN,PRO,Dario Hunter,7.0
...,...,...,...,...,...,...,...
81840,Malheur,20 - City of Jordan Valley,State House,60.0,NaN,Under Votes,1.0
81841,Malheur,21 - Rural Jordan Valley,State House,60.0,NaN,Under Votes,5.0
81842,Malheur,22 - Arock,State House,60.0,NaN,Under Votes,0.0
81843,Malheur,23 - McDermitt,State House,60.0,NaN,Under Votes,3.0


In [5]:
election_results["office"].unique()

array(['President', 'U.S. Senate', 'U.S. House', 'Secretary of State',
       'State Treasurer', 'Attorney General', 'State House',
       'County Commissioner, Position 2',
       'County Commissioner, Position 3',
       'Judge of the Supreme Court, Position 4',
       'Judge of the Court of Appeals, Position 9',
       'Judge of the Circuit Court, 23rd District, Position 4',
       'County Treasurer',
       'Linn Soil & Water Conservation District, Director, Zone 2',
       'Linn Soil & Water Conservation District, Director, Zone 4',
       'Linn Soil & Water Conservation District, Director, Zone 5',
       'Linn Soil & Water Conservation District, Director, At Large',
       'Mayor - City of Albany', 'City of Albany - Councilor, Ward IB',
       '22-183 Four Year Law Enforcement Local Option Tax Levy',
       '107 Limiting/identifing political campaign contributions',
       '108 Increase cigarette and cigar taxes',
       '109 Allows manufacture, delivery, admin. of psilocybinVot

In [6]:
office_list = ['President', 'U.S. Senate','Secretary of State',
       'State Treasurer', 'Attorney General']
election_results = election_results[election_results["office"].isin(office_list)]


In [7]:
election_results["pivot_col"] = election_results["county"] +"-" + election_results["precinct"].astype(str)

In [8]:
election_results["candidate"].unique()

drop_cand_list = ['Over Votes', 'Under Votes','undervote','over']

election_results = election_results[~election_results["candidate"].isin(drop_cand_list)]

In [9]:
election_results["cand_detailed"] = election_results["office"]+"-"+election_results["candidate"]

In [10]:
election_results["cand_detailed"].unique()

array(['President-Donald J. Trump', 'President-Joseph R. Biden',
       'President-Jo Jorgensen', 'President-Howie Hawkins',
       'President-Dario Hunter', 'President-Write-in',
       'U.S. Senate-Jo Rae Perkins', 'U.S. Senate-Jeff Merkley',
       'U.S. Senate-Ibrahim A Taher', 'U.S. Senate-Gary Dye',
       'U.S. Senate-Write-in', 'Secretary of State-Nathalie Paravicini',
       'Secretary of State-Kyle Markley',
       'Secretary of State-Kim Thatcher',
       'Secretary of State-Shemia Fagan', 'Secretary of State-Write-in',
       'State Treasurer-Michael P Marsh', 'State Treasurer-Tobias Read',
       'State Treasurer-Chris Henry', 'State Treasurer-Jeff Gudman',
       'State Treasurer-Write-in', 'Attorney General-Ellen Rosenblum',
       'Attorney General-Lars D H Hedbor',
       'Attorney General-Michael Cross', 'Attorney General-Write-in',
       'Attorney General-Write-ins', 'President-Write-ins',
       'Secretary of State-Write-ins', 'State Treasurer-Write-ins',
       'U

G20ATGDROS - Ellen Rosenblum (Democratic Party, Independent Party and Working Family Party (fusion candidate))
G20ATGRCRO - Michael Cross (Republican Party)
G20ATGLHED - Lars D. H. Hedbor (Libertarian Party)
G20ATGOWRI - Write-in Votes

In [11]:
cand_name_updates_dict = {'President-Donald J. Trump':"G20PRERTRU",
 'President-Joseph R. Biden':"G20PREDBID",
 'President-Jo Jorgensen':"G20PRELJOR", 
 'President-Howie Hawkins':"G20PREGHAW",
 'President-Dario Hunter':"G20PREPHUN", 
 'President-Write-in':"G20PREOWRI",
 'President-Jo Jorgensen /Jeremy (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / JeremyjSpike) Cohen':"G20PRELJOR",
 'President-Joseph R Biden / Kamala D Harris':"G20PREDBID",
 'President-Jo Jorgensen / Jeremy (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / Jeremy. (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / Jeremy Spike) Cohen':"G20PRELJOR",
 'President-Write-ins':"G20PREOWRI",
 'President-Howie Hawkins / Angela Walker':"G20PREGHAW",
 'President-Ballots Cast':"drop",
 'President-Writein':"G20PREOWRI",
 'President-Write-Ins':"G20PREOWRI",
 'President-Write-In':"G20PREOWRI",
 'President-Joseph R. Biden / Kamala D Harris':"G20PREDBID",

 
 'U.S. Senate-Jo Rae Perkins':"G20USSRPER",
 'U.S. Senate-Jeff Merkley':"G20USSDMER",
 'U.S. Senate-Ibrahim A Taher':"G20USSGTAH",
 'U.S. Senate-Gary Dye':"G20USSLDYE",
 'U.S. Senate-Write-in':"G20USSOWRI",
 'U.S. Senate-Write-ins':"G20USSOWRI",
 'U.S. Senate-Ibrahim A. Taher':"G20USSGTAH",
 'U.S. Senate-Ibrahim Tahir':"G20USSGTAH",
 'U.S. Senate-Gary.Dye':"G20USSLDYE",
 'U.S. Senate-Ibrahim Taher':"G20USSGTAH",
 'U.S. Senate-Ibraham Taher':"G20USSGTAH",
 'U.S. Senate-Write-In':"G20USSOWRI",
 'U.S. Senate-Write-Ins':"G20USSOWRI",
 
 'Secretary of State-Nathalie Paravicini':"G20SOSGPAR",
 'Secretary of State-Natalie Paravicini':"G20SOSGPAR",
 'Secretary of State-Nathalie Paravacini':"G20SOSGPAR",
 'Secretary of State-Nathalie Paracivini':"G20SOSGPAR",
 'Secretary of State-Kyle Markey':"G20SOSLMAR",
 'Secretary of State-Kyle Markley':"G20SOSLMAR",
 'Secretary of State-Kim Thatcher':"G20SOSRTHA",
 'Secretary of State-Shemia Fagan':"G20SOSDFAG",
 'Secretary of State-Write-in':"G20SOSOWRI",
 'Secretary of State-Write-Ins':"G20SOSOWRI",
 'Secretary of State-Write-In':"G20SOSOWRI", 
 'Secretary of State-Write-ins':"G20SOSOWRI",
 'Secretary of State-Writein':"G20SOSOWRI",
 
 'State Treasurer-Jeff Gudman':"G20TRERGUD",
 'State Treasurer-Michael P Marsh':"G20TRECMAR", 
 'State Treasurer-Michael Marsh':"G20TRECMAR",
 'State Treasurer-Michael P. Marsh':"G20TRECMAR",
 'State Treasurer-Michael P. Marsch':"G20TRECMAR",
 'State Treasurer-Tobias Read':"G20TREDREA",
 'State Treasurer-Tobias Reed':"G20TREDREA",
 'State Treasurer-Writein':"G20TREOWRI",
 'State Treasurer-Write-ins':"G20TREOWRI",
 'State Treasurer-Write-Ins':"G20TREOWRI",
 'State Treasurer-Write-In':"G20TREOWRI",
 'State Treasurer-Write-in':"G20TREOWRI",
 'State Treasurer-IND Chris Henry':"G20TREIHEN",
 'State Treasurer-Chris Henry':"G20TREIHEN",
 'State Treasurer-Chris Hen':"G20TREIHEN",
 
 'Attorney General-Ellen Rosenblum':"G20ATGDROS",
 'Attorney General-Ellen Roseblum':"G20ATGDROS",
 'Attorney General-Lars D H Hedbor':"G20ATGLHED",
 'Attorney General-Lars D.H. Hedbor':"G20ATGLHED",
 'Attorney General-Lars D. H. Hedbor':"G20ATGLHED",
 'Attorney General-Lars Hedbor':"G20ATGLHED",
 'Attorney General-Michael Cross':"G20ATGRCRO", 
 'Attorney General-Write-in':"G20ATGOWRI",
 'Attorney General-Write-ins':"G20ATGOWRI",
 'Attorney General-Writein':"G20ATGOWRI",
 'Attorney General-Write-Ins':"G20ATGOWRI",
 'Attorney General-Write-In':"G20ATGOWRI"
}

In [12]:
election_results["cand_detailed"] = election_results["cand_detailed"].map(cand_name_updates_dict).fillna(election_results["cand_detailed"])

In [13]:
election_results = election_results[election_results["cand_detailed"]!="drop"]

In [14]:
pivoted_results = pd.pivot_table(election_results,index="pivot_col",values=["votes"],aggfunc=sum,columns="cand_detailed")

In [15]:
pivoted_results.reset_index(inplace=True,drop=False)
pivoted_results.columns = pivoted_results.columns.droplevel(0)
pivoted_results.rename(columns={"":"unique_ID"},inplace=True)

In [16]:
pivoted_results.columns

Index(['unique_ID', 'G20ATGDROS', 'G20ATGLHED', 'G20ATGOWRI', 'G20ATGRCRO',
       'G20PREDBID', 'G20PREGHAW', 'G20PRELJOR', 'G20PREOWRI', 'G20PREPHUN',
       'G20PRERTRU', 'G20SOSDFAG', 'G20SOSGPAR', 'G20SOSLMAR', 'G20SOSOWRI',
       'G20SOSRTHA', 'G20TRECMAR', 'G20TREDREA', 'G20TREIHEN', 'G20TREOWRI',
       'G20TRERGUD', 'G20USSDMER', 'G20USSGTAH', 'G20USSLDYE', 'G20USSOWRI',
       'G20USSRPER'],
      dtype='object', name='cand_detailed')

In [17]:
pivoted_results = pivoted_results[pivoted_results["unique_ID"]!="Curry-nan"]

## Statewide Check

In [18]:
def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [19]:
statewide_totals_check(vest_or_20,pivoted_results,data_columns)

***Statewide Totals Check***
G20PREDBID is equal 	VEST / RDH: 1340383
G20PRERTRU is equal 	VEST / RDH: 958448
G20PRELJOR has a difference of 20.0 votes
	VEST: 41582 votes
	SOURCES: 41562.0 votes
G20PREGHAW has a difference of 8.0 votes
	VEST: 11831 votes
	SOURCES: 11823.0 votes
G20PREPHUN is equal 	VEST / RDH: 4988
G20PREOWRI is equal 	VEST / RDH: 17089
G20USSDMER is equal 	VEST / RDH: 1321047
G20USSRPER is equal 	VEST / RDH: 912814
G20USSLDYE is equal 	VEST / RDH: 42747
G20USSGTAH is equal 	VEST / RDH: 42239
G20USSOWRI is equal 	VEST / RDH: 2402
G20ATGDROS is equal 	VEST / RDH: 1264716
G20ATGRCRO is equal 	VEST / RDH: 934357
G20ATGLHED is equal 	VEST / RDH: 52087
G20ATGOWRI is equal 	VEST / RDH: 8490
G20SOSDFAG is equal 	VEST / RDH: 1146370
G20SOSRTHA is equal 	VEST / RDH: 984597
G20SOSLMAR is equal 	VEST / RDH: 62985
G20SOSGPAR is equal 	VEST / RDH: 82211
G20SOSOWRI is equal 	VEST / RDH: 2340
G20TREDREA is equal 	VEST / RDH: 1166703
G20TRERGUD is equal 	VEST / RDH: 936916
G20TREIHEN 

## County-by-County Check

In [20]:
pivoted_results["COUNTY"] = pivoted_results["unique_ID"].apply(lambda x:x.split("-")[0])

In [21]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """
    
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print()
        print(diff_counties)

In [22]:
county_totals_check(vest_or_20,pivoted_results,data_columns,"COUNTY",full_print=False)

***Countywide Totals Check***

G20PREDBID is equal across all counties
G20PRERTRU is equal across all counties
G20PRELJOR contains differences in these counties:
	Baker has a difference of 20.0 votes
		VEST: 144 votes
		SOURCES: 124.0 votes
G20PREGHAW contains differences in these counties:
	Grant has a difference of 8.0 votes
		VEST: 12 votes
		SOURCES: 4.0 votes
G20PREPHUN is equal across all counties
G20PREOWRI is equal across all counties
G20USSDMER is equal across all counties
G20USSRPER is equal across all counties
G20USSLDYE is equal across all counties
G20USSGTAH is equal across all counties
G20USSOWRI is equal across all counties
G20ATGDROS is equal across all counties
G20ATGRCRO is equal across all counties
G20ATGLHED is equal across all counties
G20ATGOWRI is equal across all counties
G20SOSDFAG is equal across all counties
G20SOSRTHA is equal across all counties
G20SOSLMAR is equal across all counties
G20SOSGPAR is equal across all counties
G20SOSOWRI is equal across all co

In [23]:
vest_or_20.groupby(["COUNTY"]).sum().to_csv("./vest_totals.csv")

## Precinct-by-Precinct Check

In [24]:
vest_or_20["unique_ID"]=vest_or_20["COUNTY"]+"-"+vest_or_20["NAME"]+" "+vest_or_20["PRECINCT"]
vest_or_20["unique_ID"] = vest_or_20["unique_ID"].str.upper()

In [25]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].str.upper()

In [26]:
vest_or_20["unique_ID"].value_counts(dropna=False)
pivoted_results["unique_ID"].value_counts(dropna=False)

KLAMATH-PRECINCT 06                  1
LINN-PRECINCT 101                    1
UMATILLA-123-MILTON-FRWTR,OUTSIDE    1
TILLAMOOK-TILLAMOOK #2               1
MARION-655                           1
                                    ..
WASHINGTON-PRECINCT 424              1
MARION-326                           1
MALHEUR-18 - HARPER                  1
WASHINGTON-PRECINCT 426              1
LINN-PRECINCT 103                    1
Name: unique_ID, Length: 1326, dtype: int64

In [27]:
join_attempt_one = pd.merge(vest_or_20,pivoted_results,on="unique_ID",how="outer",indicator=True)
join_attempt_one["_merge"].value_counts()

left_only     1331
right_only    1326
both             0
Name: _merge, dtype: int64

In [28]:

join_attempt_one[join_attempt_one["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_one[join_attempt_one["_merge"]=="right_only"].to_csv("./pivoted.csv")

In [29]:
source_vest_id_changes = pd.read_csv("./source_vest_id_changes.csv")
source_vest_id_changes_dict = dict(zip(source_vest_id_changes["election_ID"],source_vest_id_changes["vest_ID"]))

In [30]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].map(source_vest_id_changes_dict).fillna(pivoted_results["unique_ID"])

In [31]:
pivoted_results = pivoted_results.fillna(0)


In [32]:
join_attempt_two = pd.merge(vest_or_20,pivoted_results,on="unique_ID",how="outer",indicator=True)
join_attempt_two["_merge"].value_counts()

both          1326
left_only        5
right_only       0
Name: _merge, dtype: int64

In [33]:
join_attempt_two[join_attempt_two["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_two[join_attempt_two["_merge"]=="right_only"].to_csv("./pivoted.csv")

In [34]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      Nothing, only prints out an analysis
    """
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [35]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"unique_ID",print_level=0)

G20PRELJOR ...................................................BAKER-BAKER CITY 5 05 (V) ...22  (S)....2 (D):   20
G20PREGHAW .......................................................GRANT-JOHN DAY 03 (V) ....8  (S)....0 (D):    8

There are  1326  total rows
2  of these rows have election result differences
1324  of these rows are the same

The max difference between any one shared column in a row is:  20.0
The average difference is:  14.0
There are  1 precinct results with a difference greater than 10

All precincts containing differences:
['BAKER-BAKER CITY 5 05', 'GRANT-JOHN DAY 03']


In [36]:
display(pivoted_results[pivoted_results["unique_ID"]=="COOS-BANDON CITY 16"])

cand_detailed,unique_ID,G20ATGDROS,G20ATGLHED,G20ATGOWRI,G20ATGRCRO,G20PREDBID,G20PREGHAW,G20PRELJOR,G20PREOWRI,G20PREPHUN,...,G20TREDREA,G20TREIHEN,G20TREOWRI,G20TRERGUD,G20USSDMER,G20USSGTAH,G20USSLDYE,G20USSOWRI,G20USSRPER,COUNTY
231,COOS-BANDON CITY 16,1002.0,33.0,3.0,943.0,1049.0,10.0,21.0,11.0,2.0,...,946.0,61.0,3.0,919.0,1054.0,19.0,21.0,1.0,925.0,Coos


There are a few different categories of differences:
    
    1. BAKER-BAKER CITY 5 05, G20PRELJOR 20 more votes in VEST
    2. GRANT county, difference in presidential results